## Adding interactivity to human centered metrics

This notebook explores several interactive, human-centered metrics. It uses the `interact` module to provide sliders to interact with the data. For more examples of interactive data manipulation, see the [ipywidgets examples](https://github.com/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Index.ipynb). Some additional advanced examples are at the bottom of this notebook.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import matplotlib.patches as mpatches

from params.generate_all import generate_all 
from functions.vphpl_pphpl_calc import vphpl_pphpl_calc, pphpl_by_mode
from functions.plots_functions import violin, pdf_plot, cdf_plot, cdf_subplots, calc_ms_pvt 

import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline
import matplotlib as mpl
import seaborn as sns

mpl.rcParams['figure.figsize'] = [8.0, 6.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 13
mpl.rcParams['axes.labelsize'] = 13
mpl.rcParams['axes.titlesize'] = 13
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'
#mpl.rcParams['figure.subplot.wspace'] = 0.25 #Use when producing 3-wide subplots


#Color-blind friendly for line plots: https://gist.github.com/thriveth/8560036
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']


In [ ]:
#########Std dev varying functions#########

#Given binary inputs for ms, occ, fe, calculate the std dev version of PEIT1
def calculate_peit_var(ms_var, occ_var, FE_var):
    ms_bus_mod = stddev_gen(params["ms"]["bus"], ms_var)
    ms_wb_mod = stddev_gen(params["ms"]["wb"], ms_var)
    ms_pvt_mod = 1 - (ms_bus_mod + ms_wb_mod)

    pvt_peit_std=peit1_calc(ms_pvt_mod,stddev_gen(params["occ"]["pvt"], occ_var),stddev_gen(params["FE"]["pvt"], FE_var))
    bus_peit_std=peit1_calc(ms_bus_mod,stddev_gen(params["occ"]["bus"], occ_var),stddev_gen(params["FE"]["bus"], FE_var))
    wb_peit_std=peit1_calc(ms_wb_mod,1,110)
    peit_vals_std = pvt_peit_std+bus_peit_std+wb_peit_std
    return peit_vals_std

#Given binary inputs for ms, occ, fe, pmt calculate the std dev version of PEIT2
#Input of 1 means increase that parameter value by a standard deviation of 1
def calculate_peit2_var(ms_var, occ_var, FE_var, pmt_var):
    ms_bus_mod = stddev_gen(params["ms"]["bus"], ms_var)
    ms_wb_mod = stddev_gen(params["ms"]["wb"], ms_var)
    ms_pvt_mod = 1 - (ms_bus_mod + ms_wb_mod)

    pvt_peit_std=peit1_calc(ms_pvt_mod,stddev_gen(params["occ"]["pvt"], occ_var),stddev_gen(params["FE"]["pvt"], FE_var))*stddev_gen(params["pmt"]["pvt"],pmt_var)
    bus_peit_std=peit1_calc(ms_bus_mod,stddev_gen(params["occ"]["bus"], occ_var),stddev_gen(params["FE"]["bus"], FE_var))*stddev_gen(params["pmt"]["bus"],pmt_var)
    wb_peit_std=peit1_calc(ms_wb_mod,1,110)*stddev_gen(params["pmt"]["wb"],pmt_var)
    peit_vals_std = pvt_peit_std+bus_peit_std+wb_peit_std
    return peit_vals_std

#Given binary inputs for ms, occ, fe, calculate the std dev version of PPHPL
#-1 for pmt so it is an improvement
def calculate_pphpl_var(occ, ms):
    ms_bus = stddev_gen(params["ms"]["bus"], ms)
    ms_wb = stddev_gen(params["ms"]["wb"], ms)
    ms_pvt = 1 - (ms_bus + ms_wb)

    pphpl_pvt_std=pphpl_by_mode(results["vphpl"]["pvt"],params['mpw']['pvt'],stddev_gen(params["occ"]["pvt"],occ)).tolist()
    pphpl_bus_std=pphpl_by_mode(results["vphpl"]["bus"],params['mpw']['bus'],stddev_gen(params["occ"]["bus"],occ)).tolist()
    pphpl_wb_std=np.array([i/j * 1000 * params['mpw']['wb'] * 1  for i,j in zip(params["speed"]["wb"],results['tl']['wb'])]).flatten().tolist()
    pphpl_combined_std = ((pphpl_pvt_std * ms_pvt) + (pphpl_bus_std*ms_bus) + (pphpl_wb_std*ms_wb)).flatten().tolist()
     
    return pphpl_combined_std

def calculate_pphpl2_var(occ, ms, pmt):
    road_len = 20 #km
    ms_bus = stddev_gen(params["ms"]["bus"], ms)
    ms_wb = stddev_gen(params["ms"]["wb"], ms)
    ms_pvt = 1 - (ms_bus + ms_wb)

    pphpl_pvt_std=pphpl_by_mode(results["vphpl"]["pvt"],params['mpw']['pvt'],stddev_gen(params["occ"]["pvt"],occ)).tolist()*stddev_gen(road_len/params['pmt']['pvt'],pmt)
    pphpl_bus_std=pphpl_by_mode(results["vphpl"]["bus"],params['mpw']['bus'],stddev_gen(params["occ"]["bus"],occ)).tolist()*stddev_gen(road_len/params['pmt']['bus'],pmt)
    pphpl_bus_std = [0 if i < 0 else i for i in pphpl_bus_std] #Ensure negative values are zeroed
    pphpl_wb_std=np.array([i/j * 1000 * params['mpw']['wb'] * 1  for i,j in zip(params["speed"]["wb"],results['tl']['wb'])]).flatten().tolist()*stddev_gen(road_len/params['pmt']['wb'],pmt)
    pphpl_combined_std = ((pphpl_pvt_std * ms_pvt) + (pphpl_bus_std*ms_bus) + (pphpl_wb_std*ms_wb)).flatten().tolist()
    return pphpl_combined_std

## Read the parameters and generate baseline values

In [ ]:
params = generate_all()
results = vphpl_pphpl_calc(params)

# Plotting Parameters

In [ ]:
#FE subplots
cdf_subplots([params["FE"]["bus"],params["FE"]["pvt"]],['Transit bus','Private vehicle'],"Vehicle energy efficiency [km/L fuel]",['orange','blue'])

In [ ]:
#occ
cdf_subplots([params["occ"]["pvt"],params["occ"]["bus"]],['Private vehicle','Transit bus'],"Number of passengers per vehicle")

In [ ]:
#ms and reset 
cdf_subplots([params["ms"]["bus"],params["ms"]["wb"],params["ms"]["pvt"]],['Transit bus','Walk & bike','Private vehicle'],"Mode share",['orange','green','blue'])

In [ ]:
#pmt
cdf_subplots([params["pmt"]["wb"],params["pmt"]["pvt"],params["pmt"]["bus"]],['Walk & bike','Private vehicle','Transit bus'],"Trip distance [km]",['green','blue','orange'])

### PEIT Plot

In [ ]:
def peit1_calc(ms, avo, ee):
    return ms * (1/avo) * (1/ee)

In [ ]:
# Default parameter does not change anything
# 1 adds one standard deviation
# -1 removes one standard deviation
def stddev_gen(vals,stddev_ratio=1):
    stddev = np.std(vals)
    new_vals = vals + stddev_ratio * stddev
    new_vals=new_vals.flatten()
    return new_vals


def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

In [ ]:
cdf_subplots([results["peit1"]["combined"],results["peit2"]["combined"]],["PEIT1 [fuel L/km]","PEIT2 [fuel L/trip]"],'Person energy intensity for transportation',['green','blue','orange'])


# Standard Deviation Plots for PEIT

In [ ]:
#CDFs of modes all in a single plot
#Colors MUST be passed in as a list even if there's only one color
def cdf_plot(data, labels, xaxis_title = "",colors = ['blue','orange','green']):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    if (isinstance(data[0],np.ndarray) or isinstance(data[0],list)): #If we pass in a list of datasets
        for idx in range(0,len(data)):
            if labels[idx]=="Baseline":
                linewidth=3
                linestyle="--"
            else:
                linewidth=2
                linestyle="-"
            values, base = np.histogram(data[idx], bins=41)
            base[-1]= max([max(x) for x in data])
            cumulative = np.cumsum(values)
            cumulative = np.append(cumulative,100)
            
            ######
            median=np.percentile(data[idx], 50) #Using 47th percentile to make lines intersect with trends visually- unsure why 50th percentile doesn't lie on curve
            plt.axhline(y=.5, xmin=0, xmax=1, color='red',linestyle=":",linewidth=1)
            plt.axvline(x=median, ymin=0, ymax=0.5,color=colors[idx],linestyle=":",linewidth=2)
            #plt.text(median, idx/20, round(median,3), color=colors[idx], fontsize=14)
            print(median)
            ######
            
            plt.plot(base, cumulative/100, linewidth=linewidth,linestyle=linestyle,label = labels[idx],c=colors[idx])
            plt.fill_between(base, cumulative/100, 0, alpha=0.05,color=colors[idx])
    else: #There is only one dataset being passed in
        values, base = np.histogram(data, bins=41)
        base[-1]= max(data)
        cumulative = np.cumsum(values)
        cumulative = np.append(cumulative,100)
        plt.plot(base, cumulative/100, linewidth=2,label = labels,c=colors[0])
        plt.fill_between(base, cumulative/100, 0, alpha=0.05,color=colors[0])
    plt.text(np.percentile(data[idx], 90), 0.5, 'Median', color="red", fontsize=14)
    ax.set_xlabel(xaxis_title)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
#All PEIT1 std dev
cdf_plot([results['peit1']['combined'],calculate_peit_var(1,0,0),calculate_peit_var(0,1,0),calculate_peit_var(0,0,1)],
         ['Baseline','Higher mode share\n(Transit bus, Walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)','Higher fuel economy\n(Private vehicle, Transit bus)'],
         "Person energy intensity for transportation [fuel L/km]",CB_color_cycle)

In [ ]:
#All PEIT2 std dev
#-1 for trip distance because we want all of the scenarios to be improvements over the baseline
cdf_plot([results['peit2']['combined'],calculate_peit2_var(1,0,0,0),calculate_peit2_var(0,1,0,0),calculate_peit2_var(0,0,1,0),calculate_peit2_var(0,0,0,-1)],
         ['Baseline','Higher mode share\n(Transit bus, Walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)','High fuel economy\n(Private vehicle, Transit bus)','Lower avg trip distance'],
         "Person energy intensity for transportation [fuel L/trip]",CB_color_cycle)

# Standard Deviation Plots for PPHPL

In [ ]:
#PPHPL1 and 2 plots (combined standard deviation CDF curves)
cdf_plot([results['pphpl1']['combined'],calculate_pphpl_var(0,1),calculate_pphpl_var(1,0)],
         ['Baseline','Higher mode share\n(transit bus, walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)'],
         "People per hour per lane",CB_color_cycle)
cdf_plot([results['pphpl2']['combined'],calculate_pphpl2_var(0,1,0),calculate_pphpl2_var(1,0,0),calculate_pphpl2_var(0,0,1)],
         ['Baseline','Higher mode share\n(transit bus, walk & bike)','Higher occupancy\n(Private vehicle, Transit bus)','Lower trip distance'],
         "People per hour per lane (20 km)",CB_color_cycle)

### COMBINED VPHPL and PPHPL plot

In [ ]:
#COMBINED VPHPL AND PPHPL PLOT
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = []
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

add_label(plt.violinplot([results["vphpl"]["pvt"],results["vphpl"]["bus"]]), "VPHPL")    
add_label(plt.violinplot([results["pphpl1"]["pvt"],results["pphpl1"]["bus"],results["pphpl1"]["wb"]]), "PPHPL1")
add_label(plt.violinplot([results["pphpl2"]["pvt"],results["pphpl2"]["bus"],results["pphpl2"]["wb"]]), "PPHPL2 (20 km)")
ax.set_yscale("log")
xticklabels = ['Private vehicle', 'Transit bus', 'Walk & bike']
ax.set_xticks([1,2,3])
ax.set_xticklabels(xticklabels)
plt.legend(*zip(*labels), loc=2, fontsize = 12)
ax.set_ylabel('Number of vehicles or people per hour per lane')

In [ ]:
#COMBINED VPHPL AND PPHPL PLOT
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
labels = []
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

add_label(plt.violinplot([results["vphpl"]["pvt"],results["vphpl"]["bus"]]), "VPHPL")    
add_label(plt.violinplot([results["pphpl1"]["pvt"],results["pphpl1"]["bus"],results["pphpl1"]["wb"]]), "PPHPL")
ax.set_yscale("log")
xticklabels = ['Private vehicle', 'Transit bus', 'Walk & bike']
ax.set_xticks([1,2,3])
ax.set_xticklabels(xticklabels)
plt.legend(*zip(*labels), loc=2, fontsize = 12)
ax.set_ylabel('Number of vehicles or people per hour per lane')

# Standard Deviation Plots for PEIT (interactive)

In [ ]:
def explore_peit_var(ms_var, occ_var, FE_var):
    cdf_plot([results['peit1']['combined'],calculate_peit_var(ms_var, occ_var, FE_var)],['Baseline','Modified'],"PEIT1 [L/km]",['gold','red'])
    
interactive_plot = interactive(explore_peit_var, ms_var=(-3.0, 3.0,0.5), occ_var=(-3, 3, 0.5), FE_var=(-3,3,0.5))
output = interactive_plot.children[-1]
output.layout.height = '640px'
interactive_plot

### Interaction

In [ ]:
@interact(ms=widgets.FloatSlider(min=0,max=1.0, value=1.0), avo=widgets.IntSlider(min=1, max=5, step=1, value=1), ei=widgets.BoundedIntText(min=1, value=10))
def mpec_driving(ms, avo, ei):
    return 1.0 / (ms * (1/avo) * ei)

In [ ]:
def print_mpec_driving(avo):
    print(mpec_driving(ms=1.0, avo=avo, ei=10))
play = widgets.Play(
    value=1,
    min=1,
    max=5,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.FloatSlider(min=1, max=5, step=0.5, description="avo")
widgets.jslink((play, 'value'), (slider, 'value'))
controls = widgets.HBox([play, slider])
out = widgets.interactive_output(print_mpec_driving, {'avo': play})
display(controls, out)

## Additional advanced functionality examples

### Arguments that are dependent on each other

Arguments that are dependent on each other can be expressed manually using `observe`.  See the following example, where one variable is used to describe the bounds of another.  For more information, please see the [widget events example notebook](./Widget%20Events.ipynb).

In [ ]:
x_widget = widgets.FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

### Flickering and jumping output

On occasion, you may notice interact output flickering and jumping, causing the notebook scroll position to change as the output is updated. The interactive control has a layout, so we can set its height to an appropriate value (currently chosen manually) so that it will not change size as it is updated.


In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot